# Patrimônio histórico de Juiz de Fora

Para mapeamento do patrimônio histórico de Juiz de Fora, considerando os bens tombados, realizamos o download de tabela existente na Wikipedia, com o auxílio de bibliotecas do Python.

### Importamos as bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import osmnx as ox
import requests
from bs4 import BeautifulSoup
import os
from groq import Groq
from PyPDF2 import PdfReader
import tempfile
import base64
import re
from thefuzz import fuzz, process
import folium
from folium.plugins import Search, MarkerCluster 
import json

from dotenv import load_dotenv

### Baixamos a lista

In [ ]:
def extrair_bens_tombados():
    url = "https://pt.wikipedia.org/wiki/Lista_de_bens_tombados_em_Juiz_de_Fora" 
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    table = soup.find('table', {'class': 'wikitable'})
    bens_list = []

    for row in table.find_all('tr')[1:]:
        cols = row.find_all(['td', 'th'])
        if len(cols) >= 5:
            nome_bem = cols[1].get_text(strip=True)
            coord_texto = cols[3].get_text(strip=True)

            try:
                lat, lon = extrair_coordenadas(coord_texto)
                if lat is not None and lon is not None:
                    bens_list.append({
                        'Bem': nome_bem,
                        'Latitude': lat,
                        'Longitude': lon
                    })
            except Exception as e:
                print(f"Erro ao processar linha: {e}")
                continue

    return bens_list

def extrair_coordenadas(texto):
    if not texto.strip():
        return None, None

    try:
        partes = texto.split(',')
        if len(partes) < 2:
            return None, None

        lat_str = partes[0].strip()
        lon_str = partes[1].strip()

        def dms_para_decimal(dms):
            dms = dms.replace('°', ' ').replace('′', ' ').replace('″', '').strip()
            graus, minutos, segundos = map(float, dms.split()[:3])
            direcao = dms.split()[-1]

            decimal = abs(graus) + minutos / 60 + segundos / 3600
            if direcao in ['S', 'O']:
                decimal *= -1

            return decimal

        lat = dms_para_decimal(lat_str)
        lon = dms_para_decimal(lon_str)

        return lat, lon
    except Exception as e:
        print(f"Erro ao processar coordenada: {texto} - {e}")
        return None, None

# Extração dos dados
bens = extrair_bens_tombados()


### Convertemos a lista em data frame

In [ ]:
df = pd.DataFrame(bens)
df.to_csv("bens_tombados_jf.csv", index=False, sep=";", encoding="utf-8")

### Inspecionamos o data frame

In [ ]:
print(df.head())

In [ ]:
df.shape

In [ ]:
print(df.isnull().sum())

In [ ]:
df.isna().sum()

### Limpeza de dados

In [ ]:
df = df.replace(['', np.nan], np.nan).dropna()
print(df.head())

In [ ]:
df = df.drop_duplicates()
print(df.head())

In [ ]:
df = df.iloc[1:]
print(df.head())

In [ ]:
df_temp = df[~df['Bem'].str.contains('Imóvel', case=False)]
print(df_temp.head())

### Filtramos os bens com denominação genérica

In [ ]:
df_imovel = df[df['Bem'].str.contains('Imóvel', case=False)]
print(df_imovel.head())

### Manipulamos arquivo PDF com modelo llm

In [ ]:
# Carregamos a chave API da Groq
load_dotenv()

# Configuração inicial
GROQ_API_KEY = os.getenv("GROQ_API_KEY")  # Substitua pela sua chave da Groq
MODEL_NAME = "meta-llama/llama-4-maverick-17b-128e-instruct"  # Ou "gemini-1.5-pro" se disponível
PDF_PATH = "bens_tombados_17092021.pdf"

# Função para extrair texto do PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

# Função para processar o texto com o LLM
def process_with_groq(text_content):
    client = Groq(api_key=GROQ_API_KEY)
    
    prompt = f"""
    Você é um assistente especializado em extrair dados estruturados de documentos. 
    Extraia uma tabela com os bens tombados do seguinte texto, com três colunas:
    1. id (número sequencial)
    2. endereço (se disponível)
    3. nome_edificio (descrição do bem tombado)

    Formato esperado:
    id | endereço | nome_edificio
    ---|----------|-------------
    1  | Rua X, 123 | Edifício ABC

    Texto para análise:
    {text_content}
    """

    response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model=MODEL_NAME,
        temperature=0.3
    )
    
    return response.choices[0].message.content

# Função para converter a resposta em DataFrame
def parse_response_to_df(response_text):
    lines = [line.split('|') for line in response_text.strip().split('\n') if '|' in line]
    
    # Remover cabeçalho se existir
    if 'id' in lines[0][0].lower():
        lines = lines[1:]
    
    data = []
    for line in lines:
        # Limpar cada campo
        cleaned = [item.strip() for item in line]
        
        # Garantir que temos 3 colunas
        if len(cleaned) == 3:
            data.append({
                'id': cleaned[0],
                'endereco': cleaned[1],
                'nome_edificio': cleaned[2]
            })
    
    return pd.DataFrame(data)

# Fluxo principal
def main():
    # Extrair texto do PDF
    print("Extraindo texto do PDF...")
    text_content = extract_text_from_pdf(PDF_PATH)
    
    # Processar com Groq
    print("Processando com o modelo LLM...")
    response = process_with_groq(text_content)
    
    # Converter para DataFrame
    print("Convertendo para DataFrame...")
    df_funalfa = parse_response_to_df(response)
    
    # Salvar como CSV
    output_path = "bens_tombados.csv"
    df_funalfa.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Dados salvos em {output_path}")
    
    # Mostrar preview
    print("\nPreview dos dados:")
    print(df_funalfa.head())

if __name__ == "__main__":
    main()

### Extraímos uma amostra do data frame

In [ ]:
df_tombados = pd.read_csv("bens_tombados.csv")
print(df_tombados.head())

### Removemos os registros em branco

In [ ]:
df_tombados_filtrado = df_tombados.dropna()
print(df_tombados_filtrado.head())

### Atribuímos nomes específicos aos registros genéricos

In [ ]:
import pandas as pd
import re

# Função para padronizar endereços
def padronizar_endereco(endereco):
    # Remover "Imóvel à " e "nº"
    endereco = re.sub(r'Imóvel à ', '', endereco)
    endereco = re.sub(r'nº ', '', endereco)
    endereco = re.sub(r'nº', '', endereco)
    
    # Padronizar Av. vs Avenida
    endereco = re.sub(r'Av\.', 'Avenida', endereco)
    
    # Remover espaços extras e vírgulas desnecessárias
    endereco = re.sub(r'\s+', ' ', endereco).strip()
    endereco = re.sub(r',\s*$', '', endereco)
    
    # Padronizar números (ex: "nº 711" -> "711")
    endereco = re.sub(r'(\D)(\d+)', lambda m: f"{m.group(1).strip()} {m.group(2)}", endereco)
    
    return endereco.strip()

# Função para extrair a parte do endereço que usaremos para matching
def extrair_chave_endereco(endereco):
    # Extrair tipo (Rua/Avenida/Praça) e nome
    match = re.match(r'(Rua|Avenida|Praça|Estrada)\s+(.+)$', endereco)
    if match:
        tipo = match.group(1)
        resto = match.group(2)
        
        # Extrair número se existir
        num_match = re.search(r'(\d+)(?:\D|$)', resto)
        numero = num_match.group(1) if num_match else None
        
        # Construir chave
        if numero:
            return f"{tipo} {resto.split(',')[0].strip()}, {numero}"
        return f"{tipo} {resto.split(',')[0].strip()}"
    return endereco

# Atualizar df_imovel com os nomes dos edifícios
def atualizar_bens_tombados(df, df_imovel):
    # Criar cópias para não modificar os originais
    df_clean = df.copy()
    df_imovel_clean = df_imovel.copy()
    
    # Padronizar endereços em ambos DataFrames
    df_imovel_clean['endereco_padrao'] = df_imovel_clean['Bem'].apply(padronizar_endereco)
    df_imovel_clean['chave_endereco'] = df_imovel_clean['endereco_padrao'].apply(extrair_chave_endereco)
    
    df_clean['endereco_padrao'] = df_clean['endereco'].fillna('').apply(padronizar_endereco)
    df_clean['chave_endereco'] = df_clean['endereco_padrao'].apply(extrair_chave_endereco)
    
    # Criar dicionário de mapeamento (chave_endereco -> nome_edificio)
    mapeamento = df_clean.set_index('chave_endereco')['nome_edificio'].to_dict()
    
    # Atualizar o campo 'Bem' no df_imovel
    def atualizar_nome(row):
        chave = row['chave_endereco']
        if chave in mapeamento and pd.notna(mapeamento[chave]):
            return mapeamento[chave]
        return row['Bem']  # Manter o original se não encontrar
    
    df_imovel_clean['Bem'] = df_imovel_clean.apply(atualizar_nome, axis=1)
    
    # Remover colunas temporárias
    df_imovel_clean.drop(columns=['endereco_padrao', 'chave_endereco'], inplace=True)
    
    return df_imovel_clean

# Exemplo de uso:
# Supondo que df (do PDF) e df_imovel já existam como DataFrames

# Carregar os DataFrames (exemplo)
df = df_tombados_filtrado.copy()

# Atualizar o DataFrame
df_imovel_atualizado = atualizar_bens_tombados(df, df_imovel)

# Mostrar resultado
print("DataFrame atualizado:")
print(df_imovel_atualizado)

# Salvar em CSV se necessário
df_imovel_atualizado.to_csv('imoveis_atualizados.csv', index=False, encoding='utf-8-sig')

### Filtramos o dataframe atualizado

In [ ]:
df_imovel_nomeado = df_imovel_atualizado[~df_imovel_atualizado['Bem'].str.contains('Imóvel', case=False)]
print(df_imovel_nomeado)

### Unificamos os dataframes

In [ ]:
df_imovel_nomeado = df_imovel_nomeado.iloc[:-1]
df_bens = pd.concat([df_temp, df_imovel_nomeado], axis=0)
print(df_bens.head())


### Convertemos o dataframe final em dicionário

In [ ]:
bens = df_bens.to_dict('records')

### Visualizamos os dados em um mapa interativo

In [ ]:

if not bens:
    print("Nenhum bem foi extraído. Verifique a estrutura da tabela na página.")
else:
    # Criar mapa base centrado em Juiz de Fora
    museu_mapa = folium.Map(location=[-21.7625, -43.35], zoom_start=13)

    # Estrutura GeoJSON para armazenar os bens (apenas para busca)
    obras_geojson = {
        "type": "FeatureCollection",
        "features": []
    }

    # Criar FeatureGroup para os marcadores visíveis
    bens_group = folium.FeatureGroup(name="Bens Tombados", show=True)

    # Preencher o GeoJSON e adicionar marcadores visíveis
    for item in bens:
        nome = item['Bem']
        lat = item['Latitude']
        lon = item['Longitude']

        if lat is not None and lon is not None:
            # Adicionar ao GeoJSON (para busca)
            obras_geojson["features"].append({
                "type": "Feature",
                "properties": {"nome": nome},
                "geometry": {
                    "type": "Point",
                    "coordinates": [lon, lat]
                }
            })

            # Adicionar marcador visível ao FeatureGroup
            folium.Marker(
                location=[lat, lon],
                popup=nome,
                icon=folium.Icon(color="orange", icon="monument", prefix="fa")
            ).add_to(bens_group)

    # Adicionar o FeatureGroup ao mapa
    bens_group.add_to(museu_mapa)

    # Criar camada GeoJSON oculta apenas para busca
    geojson_layer = folium.GeoJson(
        obras_geojson,
        name="GeoJSON Busca",
        style_function=lambda x: {
            'fillOpacity': 0,  # Totalmente transparente
            'opacity': 0,      # Totalmente transparente
            'radius': 0        # Tamanho zero
        },
        marker=folium.Circle(radius=0),  # Marcador invisível
        control=False  # Não aparece no controle de camadas
    ).add_to(museu_mapa)

    # Configurar o plugin de busca na camada GeoJSON oculta
    search_plugin = Search(
        layer=geojson_layer,
        search_label="nome",
        placeholder="Buscar obra...",
        collapsed=False,
        position='topleft'
    ).add_to(museu_mapa)

    # Adicionar controle de camadas
    folium.LayerControl().add_to(museu_mapa)

    # Salvar mapa
    museu_mapa.save("mapa_bens_tombados_jf.html")
    print("Mapa salvo como 'mapa_bens_tombados_jf.html'")

### Agrupamos os dados em um mapa interativo

In [ ]:
if not bens:
    print("Nenhum bem foi extraído. Verifique a estrutura da tabela na página.")
else:
    # Criar mapa base centrado em Juiz de Fora
    museu_mapa = folium.Map(location=[-21.7625, -43.35], zoom_start=13)

    # Criar um MarkerCluster para agrupar os marcadores
    marker_cluster = MarkerCluster(
        name="Bens Tombados",
        overlay=True,
        control=True,
        options={'maxClusterRadius': 40}
    ).add_to(museu_mapa)

    # Estrutura GeoJSON para armazenar os bens (apenas para busca)
    obras_geojson = {
        "type": "FeatureCollection",
        "features": []
    }

    # Preencher o GeoJSON e adicionar marcadores ao cluster
    for item in bens:
        nome = item['Bem']
        lat = item['Latitude']
        lon = item['Longitude']

        if lat is not None and lon is not None:
            # Adicionar ao GeoJSON (para busca)
            obras_geojson["features"].append({
                "type": "Feature",
                "properties": {"nome": nome},
                "geometry": {
                    "type": "Point",
                    "coordinates": [lon, lat]
                }
            })

            # Adicionar marcador ao cluster
            folium.Marker(
                location=[lat, lon],
                popup=nome,
                icon=folium.Icon(color="orange", icon="monument", prefix="fa")
            ).add_to(marker_cluster)

    # Criar camada GeoJSON oculta apenas para busca
    geojson_layer = folium.GeoJson(
        obras_geojson,
        name="GeoJSON Busca",
        style_function=lambda x: {
            'fillOpacity': 0,  # Totalmente transparente
            'opacity': 0,      # Totalmente transparente
            'radius': 0        # Tamanho zero
        },
        marker=folium.Circle(radius=0),  # Marcador invisível
        control=False  # Não aparece no controle de camadas
    ).add_to(museu_mapa)

    # Configurar o plugin de busca na camada GeoJSON oculta
    search_plugin = Search(
        layer=geojson_layer,
        search_label="nome",
        placeholder="Buscar obra...",
        collapsed=False,
        position='topleft'
    ).add_to(museu_mapa)

    # Adicionar controle de camadas
    folium.LayerControl().add_to(museu_mapa)

    # Salvar mapa
    museu_mapa.save("cluster_bens_tombados_jf.html")
    print("Mapa salvo como 'cluster_bens_tombados_jf.html'")

### Filtramos os pontos dentro do polígono de Juiz de Fora

In [ ]:
# Obter os limites de Juiz de Fora
place = 'Juiz de Fora, MG, Brasil'
gdf = ox.geocoder.geocode_to_gdf(place)

# Extrair as coordenadas da bounding box (bbox)
bbox = gdf.iloc[0][['bbox_west', 'bbox_south', 'bbox_east', 'bbox_north']].values

# Criar o dicionário no formato desejado
JF_BOUNDS = {
    'min_lat': float(bbox[1]),  # bbox_south
    'max_lat': float(bbox[3]),  # bbox_north
    'min_lon': float(bbox[0]),  # bbox_west
    'max_lon': float(bbox[2])   # bbox_east
}

def dentro_dos_limites(lat, lon):
    return (JF_BOUNDS['min_lat'] <= lat <= JF_BOUNDS['max_lat'] and
            JF_BOUNDS['min_lon'] <= lon <= JF_BOUNDS['max_lon'])


if not bens:
    print("Nenhum bem foi extraído. Verifique a estrutura da tabela na página.")
else:
    # Filtrar bens que estão dentro dos limites de JF
    bens_filtrados = [item for item in bens 
                     if item['Latitude'] is not None 
                     and item['Longitude'] is not None
                     and dentro_dos_limites(item['Latitude'], item['Longitude'])]
    
    # Criar mapa base centrado em Juiz de Fora
    museu_mapa = folium.Map(location=[-21.7625, -43.35], zoom_start=13)

    # Criar um MarkerCluster para agrupar os marcadores
    marker_cluster = MarkerCluster(
        name="Bens Tombados",
        overlay=True,
        control=True,
        options={'maxClusterRadius': 40}
    ).add_to(museu_mapa)

    # Estrutura GeoJSON para armazenar os bens (apenas para busca)
    obras_geojson = {
        "type": "FeatureCollection",
        "features": []
    }

    # Preencher o GeoJSON e adicionar marcadores ao cluster
    for item in bens_filtrados:
        nome = item['Bem']
        lat = item['Latitude']
        lon = item['Longitude']

        # Adicionar ao GeoJSON (para busca)
        obras_geojson["features"].append({
            "type": "Feature",
            "properties": {"nome": nome},
            "geometry": {
                "type": "Point",
                "coordinates": [lon, lat]
            }
        })

        # Adicionar marcador ao cluster
        folium.Marker(
            location=[lat, lon],
            popup=nome,
            icon=folium.Icon(color="orange", icon="monument", prefix="fa")
        ).add_to(marker_cluster)

    # Criar camada GeoJSON oculta apenas para busca
    geojson_layer = folium.GeoJson(
        obras_geojson,
        name="GeoJSON Busca",
        style_function=lambda x: {
            'fillOpacity': 0,
            'opacity': 0,
            'radius': 0
        },
        marker=folium.Circle(radius=0),
        control=False
    ).add_to(museu_mapa)

    # Configurar o plugin de busca
    search_plugin = Search(
        layer=geojson_layer,
        search_label="nome",
        placeholder="Buscar obra...",
        collapsed=False,
        position='topleft'
    ).add_to(museu_mapa)

    # Adicionar controle de camadas
    folium.LayerControl().add_to(museu_mapa)

    # Salvar mapa
    museu_mapa.save("cluster_bens_tombados_jf_1.html")
    print(f"Mapa salvo com {len(bens_filtrados)} bens tombados dentro dos limites de JF")